In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh
import qutip as qt
def annihilation(d):
    return np.diag(np.sqrt(np.arange(1, d)), 1)

def creation(d):
    return annihilation(d).T

In [34]:
# Parameters
def state_index(index,dim):
    n,k = index
    N,K = dim
    return n*K+k
def Hamiltonian(omega3):
    d1 = d2 = 5
    omega1 = 5 * 2 * np.pi
    delta1 = -0.24 * 2 * np.pi
    delta3 = -0.14 * 2 * np.pi
    g13 = 0.11 * 2 * np.pi

    # Create operators
    a1, a3 = [annihilation(d) for d in (d1, d2)]
    a1d, a3d = [creation(d) for d in (d1, d2)]
    I1,  I3 = [np.eye(d) for d in (d1, d2)]

    # Construct Hamiltonians
    H1 = omega1 * a1d @ a1 + delta1/2 * a1d @ a1d @ a1 @ a1
    H3 = omega3 * a3d @ a3 + delta3/2 * a3d @ a3d @ a3 @ a3

    H1 = np.kron(I3, H1)
    H3 = np.kron(H3, I1)

    Hint13 = g13 * np.kron(a3d + a3,  a1d + a1)

    # Total Hamiltonian
    H = H1 + H3 + Hint13
    
    Hc = np.kron(a3d + a3,  I1)
    noise_op = np.kron(a3d @ a3,  I1)
    return H, Hc, noise_op
 

In [35]:
def find_optimal_k(A, B, D):
    # Define a large initial minimum difference
    min_diff = float('inf')
    optimal_k = None
    
    # Iterate over a range of possible k values
    # The range can be adjusted based on expected size of k or other insights you have about your problem
    for k in range(-1000, 1000):
        # Calculate the difference for this value of k
        diff = abs(A - (B + k * D))
        
        # If this is the smallest difference we've found so far, update min_diff and optimal_k
        if diff < min_diff:
            min_diff = diff
            optimal_k = k
            
    return optimal_k
# Function to calculate overlap (you might use inner product, fidelity, etc.)
def calculate_overlap(state1, state2):
    return abs((state1.dag() * state2))**2


In [36]:
def calculate_floquet_energies(A, omega,omega3):
    # Define system parameters

    H0, Hc, noise_op = Hamiltonian(omega3)
    dim = [5,5]
    index01 = state_index([0,1], dim)
    index10 = state_index([1,0], dim)
    index20 = state_index([2,0], dim)

    omega1 = np.diag(H0)[index01]

    H0 = qt.Qobj(H0)
    Hc = qt.Qobj(Hc)

    T = (2 * np.pi) / omega

    # Define the Hamiltonian
    H = [H0, [Hc, lambda t, args: A * np.cos(omega * t)]]

    # Set up the Floquet solver
    floquet_basis = qt.FloquetBasis(H, T,)

    # Compute Floquet modes and energies
    f_modes = floquet_basis.mode(0)
    f_energies = floquet_basis.e_quasi

    # Define basis states
    basis_states = [qt.basis(H0.dims[0][0], 0), 
                    qt.basis(H0.dims[0][0], index01),
                    qt.basis(H0.dims[0][0], index10),
                    qt.basis(H0.dims[0][0], index20),]

    # Find Floquet states with maximum overlap
    max_overlap_indices = [-1] * 4
    max_overlaps = [0] * 4
    for f_index, f_state in enumerate(f_modes):
        for b_index, b_state in enumerate(basis_states):
            overlap = calculate_overlap(f_state, b_state)
            if overlap > max_overlaps[b_index]:
                max_overlaps[b_index] = overlap
                max_overlap_indices[b_index] = f_index

    # Calculate energies
    energy00 = f_energies[max_overlap_indices[0]] / (2 * np.pi)

    energy01 = f_energies[max_overlap_indices[1]] / (2 * np.pi)
    k = find_optimal_k(omega1 / (2 * np.pi), energy01, omega / (2 * np.pi))
    energy01 = energy01 + k * omega / (2 * np.pi) - energy00

    return energy01*2*np.pi, f_modes[max_overlap_indices[0]], f_modes[max_overlap_indices[1]],max_overlap_indices

In [37]:
def calculate_derivatives(A, omegad,omega3):
    energy01, _, _,_ = calculate_floquet_energies(A, omegad, omega3)
    epsilon = 1e-6
    energy01_, _, _,_ = calculate_floquet_energies(A, omegad, omega3 + epsilon)
    der1 = (energy01_ - energy01) / epsilon
    return der1, omegad/2/np.pi
from scipy.optimize import minimize_scalar


omega3 = 3.47286432160804*2*np.pi
A = 0.001*2*np.pi
def objective(omegad):
    der, _ = calculate_derivatives(A, omegad, omega3)
    return abs(der)

result = minimize_scalar(objective, bounds=(omega3-1, 4*2*np.pi), method='bounded')
optimal_omegad = result.x
print(f"Optimal omegad: {optimal_omegad/2/np.pi}")


Optimal omegad: 3.469080656026792


In [64]:
from auxiliary_func import *
def noise_spectrum(w):
    # noise power spectrum
    A = 0.9*2*np.pi*1e-3
    factor = np.sqrt(np.abs(np.log(2*np.pi*1e-5)*2))
    return A*factor * (w==0)
# def noise_spectrum(w):
#     """
#     Noise power spectrum: returns an array/batch result if w is an array,
#     or a single float if w is a scalar.
#     """
#     gamma1 = 1e-3
#     w_arr = np.asarray(w)  # Convert to NumPy array (even if scalar)
    
#     # Initialize result with zeros of the same shape
#     result = np.zeros_like(w_arr, dtype=float)
    
#     # Mask for values where w > 10
#     mask = w_arr >=0.000*2*np.pi
#     # For those values, apply 4*gamma1 / w
#     result[mask] = 4 * gamma1
#     return result
def matrix_element(A,omegad, noise_spectrum):
    omega3 = 3.47286432160804*2*np.pi
   
    _, dressed_00, dressed_01,index = calculate_floquet_energies(A, omegad,omega3)

    H0, Hc, noise_op = Hamiltonian(omega3)
    H0 = qt.Qobj(H0)
    Hc = qt.Qobj(Hc)
    noise_op = qt.Qobj(noise_op)
    T     = 2*np.pi / omegad
    H = [H0, [Hc, lambda t, args: A * np.cos(omegad * t)]]
    floquet_basis = qt.FloquetBasis(H, T,)
    f_modes = floquet_basis.mode(0)
    # print(f_modes[index[2]])
    temperature = 0e-3 # unit K 
    h = 6.626e-34 
    kB = 1.38e-23 
    w_th = temperature * (kB / h) * 2 * np.pi * 1e-9    
    A,w = obtain_Aw(H, [noise_op], [noise_spectrum], T, w_th)
    dephasing = w.to_array()[index[0],index[1]]
    excitation1 = w.to_array()[index[0],index[2]]/2/np.pi
    excitation2 = A.to_array()[index[0],index[3]]
    return dephasing, excitation1, excitation2

    

In [65]:
import numpy as np
import pandas as pd
import plotly.express as px

# Define the range of omegad values
omegad_values = omega_range = np.linspace(3.385*2*np.pi,3.52*2*np.pi,200)

# Initialize lists to store the results
dephasing_results = []
excitation1_results = []
excitation2_results = []
# Perform the sweep
A = 0.001 * 2 * np.pi
for omegad in omegad_values:
    dephasing, excitation1, excitation2 = matrix_element(A, omegad, noise_spectrum)
    dephasing_results.append(np.abs(dephasing))
    excitation1_results.append(np.abs(excitation1))
    excitation2_results.append(np.abs(excitation2))
# Convert omegad values back to GHz for plotting
omegad_values_ghz = omegad_values / (2 * np.pi)

# Create a DataFrame to hold the data
data = pd.DataFrame({
    'omegad (GHz)': omegad_values_ghz,
    'qubit Dephasing': dephasing_results,
    'ancilla Excitation01': excitation1_results,
    'ancilla Excitation02': excitation2_results
})

# Melt the DataFrame to long format
data_melted = data.melt(id_vars=['omegad (GHz)'], value_vars=['qubit Dephasing', 'ancilla Excitation01','ancilla Excitation02'], 
                        var_name='Type', value_name='Value')

# Plot both Dephasing and Excitation on the same plot with log scale
fig = px.line(data_melted, x='omegad (GHz)', y='Value', color='Type', 
              title='Dephasing and Excitation vs omegad (Log Scale)')

# Update the layout to use log scale for the y-axis
fig.update_layout(yaxis_type="log")

fig.show()

In [66]:
0.024*5e-3

0.00012

In [40]:
129/4.37

29.51945080091533

In [26]:
A = 0.9*2*np.pi*1e6
1/(np.sqrt(np.abs(np.log(1e-5*2*np.pi))*2)*A/2/np.pi)

2.5258986431874707e-07

In [57]:
A = 0.9*2*np.pi*1e-3
factor = np.sqrt(np.abs(np.log(2*np.pi*1e-5)*2))
A*factor

0.024875049219119646